# 보스턴 집값 예측

## 패키지 로딩

In [6]:
# 경고 메세지 뜨지 않게 하는 법
import warnings
warnings.filterwarnings(action='ignore') # action='default' : 원상복귀됨

In [5]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# 랜덤포레스트
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score # 모델 평가지표

import pandas as pd
import numpy as np

## 데이터 로드 및 확인

In [7]:
x, y = load_boston(return_X_y=True)

## 학습/평가용 데이터 분할

In [12]:
scaled_data = StandardScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(scaled_data, y,
                                                    test_size=0.3, 
                                                    random_state=10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((354, 13), (152, 13), (354,), (152,))

## 모델 생성, 학습, 예측, 검증

In [18]:
# 모델 생성
# n_estimators : 생성할 tree의 개수
n_estimators = 20
model = RandomForestRegressor(n_estimators=n_estimators, random_state=0)

# 모델 학습
model.fit(x_train, y_train)

# 예측
y_hat = model.predict(x_test)

# 모델 검증
print(f'n_estimators: {n_estimators}개')
print(f'MSE: {mean_squared_error(y_test, y_hat):.3f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 Score: {r2_score(y_test, y_hat):.3f}')
# R2 Score : LinearRegression에서는 R2 Score가 0.7이었다.
# RandomForest가 더 좋은 예측 결과를 보여준다.

n_estimators: 20개
MSE: 12.050
RMSE: 3.471
R2 Score: 0.877


## 하이퍼 파라미터 튜닝

In [26]:
from sklearn.model_selection import GridSearchCV


params = [ {'n_estimators': [3,10,20,30,40,50],
           'max_features': [2,4,6,8]},
           {'n_estimators': [3,10],
           'max_features': [2,3,4],
           'bootstrap': [False]} ]
# max_features : 의사결정나무의 최적의 노드 분할을 위해 사용하는 피처의 수(default : None)
# bootstrap : True면 복원 추출 / False면 비복원 추출
model = RandomForestRegressor()
gs = GridSearchCV(model, params, cv=5, scoring='r2')
# cv : 폴드의 갯수 (DEFAULT는 5회)
# scoring : 평가지표의 이름을 리스트로 전달
gs.fit(x_train, y_train)
print('best params:', gs.best_params_)
# gs.best_params_ : 최적의 파라미터 조합 결과 반환

best params: {'bootstrap': False, 'max_features': 4, 'n_estimators': 10}


## 튜닝 후 모델 검증

In [27]:
y_hat = gs.best_estimator_.predict(x_test)
# best_estimator_ : 최적의 결과를 보여주는 파라미터 조합으로 만들어진 분류 모형

# 모델 검증
print('best params:', gs.best_params_)
print(f'MSE: {mean_squared_error(y_test, y_hat):.3f}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_hat)):.3f}')
print(f'R2 Score: {r2_score(y_test, y_hat):.3f}')

best params: {'bootstrap': False, 'max_features': 4, 'n_estimators': 10}
MSE: 9.993
RMSE: 3.161
R2 Score: 0.898
